![sql](img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. YOu need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use `JOIN` to merge tables along logical columns
    - Grouping Data with SQL, inlcuding `HAVING`
- Goal 5: Convert SQL to pandas

## Goal 1: Summarize

To date the data we've seen has looked like [this.](https://data.austintexas.gov/Health-and-Community-Services/Austin-Animal-Center-Intakes/wter-evkm)

That is how we need data to look to run analysis and build models.<br>
But it doesn't _live_ there in it's native state.

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

## What does it mean to **Engineer Data**?

Let's start with a basic scenario:<br>
You are HR. You have *no* current database. 
How would you set it up?

Data needs considerations:
- What you want to store
- What "views" you anticipate wanting in the future

Structure considerations:
- Speed of retrieval
- How much data you are accessing
- How much you are storing

![etl](img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
### SQL

[History of SQL](https://www.businessnewsdaily.com/5804-what-is-sql.html)

> Future users of large data banks must be protected from having to know how the data is organized in the machine (the internal representation). A prompting service which supplies such information is not a satisfactory solution. Activities of users at terminals and most application programs should remain unaffected when the internal representation of data is changed and even when some aspects of the external representation are changed. Changes in data representation will often be needed as a result of changes in query, update, and report traffic and natural growth in the types of stored information.
.
.
.<br>
<br>
In Section 1 a relational model of data is proposed as a basis for protecting users of formatted data systems from the potentially disruptive changes in data representation caused by growth in the data bank and changes in traffic.

[The original SQL paper from the 1970s.](https://www.seas.upenn.edu/~zives/03f/cis550/codd.pdf)

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

***
### SQLite

<img src="img/SQLite-Python.jpg" width=500>

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## But what about connecting to database servers?

To connect to other database types there are many different libraries:
- Oracle: [cx_Oracle](https://oracle.github.io/python-cx_Oracle/)
- MySQL: [MySQL-python](https://github.com/farcepest/MySQLdb1)
- PostgreSQL: [Psycopg2](http://initd.org/psycopg/docs/)
- Microsoft SQL Server: [pymssql](http://www.pymssql.org/en/stable/)

These all implement [PEP 249: DB API v2](https://www.python.org/dev/peps/pep-0249/)

### The Structure of a SQL Query

<img src='img/sql_statement.jpg'/>

***
## Goal 3: Get going with sqlite!

In [1]:
import sqlite3

#### Load a DB object with `connect` and `cursor`

In [2]:
con = sqlite3.connect('data/flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [3]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('airports',), ('airlines',), ('routes',)]


In [4]:
cursor.description

(('name', None, None, None, None, None, None),)

In [5]:
cursor.execute("SELECT name, type FROM sqlite_master;")
print(cursor.fetchall())

[('airports', 'table'), ('ix_airports_index', 'index'), ('airlines', 'table'), ('ix_airlines_index', 'index'), ('routes', 'table'), ('ix_routes_index', 'index')]


In [6]:
cursor.execute("SELECT * FROM sqlite_master;")
results = cursor.fetchall()
for res in results:
    print(res)

('table', 'airports', 'airports', 2, 'CREATE TABLE airports (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [city] TEXT,\n  [country] TEXT,\n  [code] TEXT,\n  [icao] TEXT,\n  [latitude] TEXT,\n  [longitude] TEXT,\n  [altitude] TEXT,\n  [offset] TEXT,\n  [dst] TEXT,\n  [timezone] TEXT\n)')
('index', 'ix_airports_index', 'airports', 3, 'CREATE INDEX ix_airports_index ON airports ([index])')
('table', 'airlines', 'airlines', 945, 'CREATE TABLE airlines (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [alias] TEXT,\n  [iata] TEXT,\n  [icao] TEXT,\n  [callsign] TEXT,\n  [country] TEXT,\n  [active] TEXT\n)')
('index', 'ix_airlines_index', 'airlines', 946, 'CREATE INDEX ix_airlines_index ON airlines ([index])')
('table', 'routes', 'routes', 1393, 'CREATE TABLE routes (\n[index] INTEGER,\n  [airline] TEXT,\n  [airline_id] TEXT,\n  [source] TEXT,\n  [source_id] TEXT,\n  [dest] TEXT,\n  [dest_id] TEXT,\n  [codeshare] TEXT,\n  [stops] TEXT,\n  [equipment] TEXT\n)')
('index', 'ix_routes_i

#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [7]:
cursor.execute("SELECT * FROM airports").fetchone()

(0,
 '1',
 'Goroka',
 'Goroka',
 'Papua New Guinea',
 'GKA',
 'AYGA',
 '-6.081689',
 '145.391881',
 '5282',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [8]:
cursor.execute("SELECT * FROM routes").fetchone()

(0, '2B', '410', 'AER', '2965', 'KZN', '2990', None, '0', 'CR2')

In [9]:
cursor.fetchone()

(1, '2B', '410', 'ASF', '2966', 'KZN', '2990', None, '0', 'CR2')

In [10]:
cursor.fetchmany(4)

[(2, '2B', '410', 'ASF', '2966', 'MRV', '2962', None, '0', 'CR2'),
 (3, '2B', '410', 'CEK', '2968', 'KZN', '2990', None, '0', 'CR2'),
 (4, '2B', '410', 'CEK', '2968', 'OVB', '4078', None, '0', 'CR2'),
 (5, '2B', '410', 'DME', '4029', 'KZN', '2990', None, '0', 'CR2')]

#### Use description

In [11]:
cursor.description

(('index', None, None, None, None, None, None),
 ('airline', None, None, None, None, None, None),
 ('airline_id', None, None, None, None, None, None),
 ('source', None, None, None, None, None, None),
 ('source_id', None, None, None, None, None, None),
 ('dest', None, None, None, None, None, None),
 ('dest_id', None, None, None, None, None, None),
 ('codeshare', None, None, None, None, None, None),
 ('stops', None, None, None, None, None, None),
 ('equipment', None, None, None, None, None, None))

#### Or use `Pragma`
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`

In [12]:
cursor.execute("PRAGMA table_info(airports)")
info = cursor.fetchall()
info

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'id', 'TEXT', 0, None, 0),
 (2, 'name', 'TEXT', 0, None, 0),
 (3, 'city', 'TEXT', 0, None, 0),
 (4, 'country', 'TEXT', 0, None, 0),
 (5, 'code', 'TEXT', 0, None, 0),
 (6, 'icao', 'TEXT', 0, None, 0),
 (7, 'latitude', 'TEXT', 0, None, 0),
 (8, 'longitude', 'TEXT', 0, None, 0),
 (9, 'altitude', 'TEXT', 0, None, 0),
 (10, 'offset', 'TEXT', 0, None, 0),
 (11, 'dst', 'TEXT', 0, None, 0),
 (12, 'timezone', 'TEXT', 0, None, 0)]

#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

In [13]:
cursor.execute("PRAGMA table_info(airlines)")
info = cursor.fetchall()
info

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'id', 'TEXT', 0, None, 0),
 (2, 'name', 'TEXT', 0, None, 0),
 (3, 'alias', 'TEXT', 0, None, 0),
 (4, 'iata', 'TEXT', 0, None, 0),
 (5, 'icao', 'TEXT', 0, None, 0),
 (6, 'callsign', 'TEXT', 0, None, 0),
 (7, 'country', 'TEXT', 0, None, 0),
 (8, 'active', 'TEXT', 0, None, 0)]

In [14]:
cursor.execute("SELECT * FROM airlines")
info =cursor.fetchall()
info

[(0, '1', 'Private flight', '\\N', '-', None, None, None, 'Y'),
 (1, '2', '135 Airways', '\\N', None, 'GNL', 'GENERAL', 'United States', 'N'),
 (2, '3', '1Time Airline', '\\N', '1T', 'RNX', 'NEXTIME', 'South Africa', 'Y'),
 (3,
  '4',
  '2 Sqn No 1 Elementary Flying Training School',
  '\\N',
  None,
  'WYT',
  None,
  'United Kingdom',
  'N'),
 (4, '5', '213 Flight Unit', '\\N', None, 'TFU', None, 'Russia', 'N'),
 (5,
  '6',
  '223 Flight Unit State Airline',
  '\\N',
  None,
  'CHD',
  'CHKALOVSK-AVIA',
  'Russia',
  'N'),
 (6,
  '7',
  '224th Flight Unit',
  '\\N',
  None,
  'TTF',
  'CARGO UNIT',
  'Russia',
  'N'),
 (7,
  '8',
  '247 Jet Ltd',
  '\\N',
  None,
  'TWF',
  'CLOUD RUNNER',
  'United Kingdom',
  'N'),
 (8, '9', '3D Aviation', '\\N', None, 'SEC', 'SECUREX', 'United States', 'N'),
 (9,
  '10',
  '40-Mile Air',
  '\\N',
  'Q5',
  'MLA',
  'MILE-AIR',
  'United States',
  'Y'),
 (10, '11', '4D Air', '\\N', None, 'QRT', 'QUARTET', 'Thailand', 'N'),
 (11,
  '12',
  '611897 

In [15]:
cursor.execute("PRAGMA table_info(airports)")
info = cursor.fetchall()
info

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'id', 'TEXT', 0, None, 0),
 (2, 'name', 'TEXT', 0, None, 0),
 (3, 'city', 'TEXT', 0, None, 0),
 (4, 'country', 'TEXT', 0, None, 0),
 (5, 'code', 'TEXT', 0, None, 0),
 (6, 'icao', 'TEXT', 0, None, 0),
 (7, 'latitude', 'TEXT', 0, None, 0),
 (8, 'longitude', 'TEXT', 0, None, 0),
 (9, 'altitude', 'TEXT', 0, None, 0),
 (10, 'offset', 'TEXT', 0, None, 0),
 (11, 'dst', 'TEXT', 0, None, 0),
 (12, 'timezone', 'TEXT', 0, None, 0)]

In [16]:
cursor.execute("SELECT * FROM airports").fetchone()

(0,
 '1',
 'Goroka',
 'Goroka',
 'Papua New Guinea',
 'GKA',
 'AYGA',
 '-6.081689',
 '145.391881',
 '5282',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [17]:
cursor.execute("PRAGMA table_info(routes)")
info = cursor.fetchall()
info

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'airline', 'TEXT', 0, None, 0),
 (2, 'airline_id', 'TEXT', 0, None, 0),
 (3, 'source', 'TEXT', 0, None, 0),
 (4, 'source_id', 'TEXT', 0, None, 0),
 (5, 'dest', 'TEXT', 0, None, 0),
 (6, 'dest_id', 'TEXT', 0, None, 0),
 (7, 'codeshare', 'TEXT', 0, None, 0),
 (8, 'stops', 'TEXT', 0, None, 0),
 (9, 'equipment', 'TEXT', 0, None, 0)]

In [18]:
cursor.execute("SELECT * FROM routes").fetchone()

(0, '2B', '410', 'AER', '2965', 'KZN', '2990', None, '0', 'CR2')

***
## Goal 4: Use basic SQL commands 
- Construct SQL queries
- Use JOIN to merge tables along logical columns
- Grouping Data with SQL

### Construct SQL queries

**Select**

**From**

**Where**

**Order by**

**Limit**

#### Options for each:

**Select**:  `distinct`, using `as` to rename columns, single number aggregates

**From:** also uses aliasing with `as`

**Where**: equals is only one `=`, `between`, `in`, wildcards `%`, `AND`, `OR`, `NOT`

**Order by**: `asc` and `desc`

**Limit**:  #

**Task**: 
- Select only active airlines in the UK from the airlines table
- Select the unique list of countries with airports

In [19]:
# cursor.execute("""
# SELECT *
# FROM airports where id= '2965'
# """)
# cursor.fetchmany(5)

In [20]:
# cursor.execute("""
# SELECT *
# FROM routes
# """)
# cursor.fetchmany(5)

In [21]:
cursor.execute("""
SELECT name
FROM airlines
WHERE active = 'Y'
  AND country = 'United Kingdom';
""")
cursor.fetchmany(5)

[('Astraeus',),
 ('Air Southwest',),
 ('Aurigny Air Services',),
 ('Air Wales',),
 ('AD Aviation',)]

In [22]:
#Select the unique list of countries with airports
cursor.execute("""
SELECT distinct country
FROM airports

""")
cursor.fetchall()

[('Papua New Guinea',),
 ('Greenland',),
 ('Iceland',),
 ('Canada',),
 ('Algeria',),
 ('Benin',),
 ('Burkina Faso',),
 ('Ghana',),
 ("Cote d'Ivoire",),
 ('Nigeria',),
 ('Niger',),
 ('Tunisia',),
 ('Togo',),
 ('Belgium',),
 ('Germany',),
 ('United States',),
 ('Estonia',),
 ('Finland',),
 ('United Kingdom',),
 ('Guernsey',),
 ('Jersey',),
 ('Isle of Man',),
 ('Falkland Islands',),
 ('Netherlands',),
 ('Ireland',),
 ('Denmark',),
 ('Faroe Islands',),
 ('Luxembourg',),
 ('Norway',),
 ('Poland',),
 ('Sweden',),
 ('South Africa',),
 ('Botswana',),
 ('Congo (Brazzaville)',),
 ('Congo (Kinshasa)',),
 ('Swaziland',),
 ('Central African Republic',),
 ('Equatorial Guinea',),
 ('Saint Helena',),
 ('Mauritius',),
 ('British Indian Ocean Territory',),
 ('Cameroon',),
 ('Zambia',),
 ('Comoros',),
 ('Mayotte',),
 ('Reunion',),
 ('Madagascar',),
 ('Angola',),
 ('Puerto Rico',),
 ('Gabon',),
 ('Sao Tome and Principe',),
 ('Mozambique',),
 ('Seychelles',),
 ('Chad',),
 ('Zimbabwe',),
 ('Malawi',),
 ('Le

### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

<img src="img/venn.png" width=550>

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [23]:
cursor.execute(""" select ao.latitude, ao.longitude, ro.*
from routes ro
inner join airports ao on ao.id = ro.source_id
inner join airports aa on aa.id = ro.dest_id;
""")
#print(*cursor.description, sep='\n')

cursor.fetchmany(5)



[('43.449928',
  '39.956589',
  0,
  '2B',
  '410',
  'AER',
  '2965',
  'KZN',
  '2990',
  None,
  '0',
  'CR2'),
 ('46.283333',
  '48.006278',
  1,
  '2B',
  '410',
  'ASF',
  '2966',
  'KZN',
  '2990',
  None,
  '0',
  'CR2'),
 ('46.283333',
  '48.006278',
  2,
  '2B',
  '410',
  'ASF',
  '2966',
  'MRV',
  '2962',
  None,
  '0',
  'CR2'),
 ('55.305836',
  '61.503333',
  3,
  '2B',
  '410',
  'CEK',
  '2968',
  'KZN',
  '2990',
  None,
  '0',
  'CR2'),
 ('55.305836',
  '61.503333',
  4,
  '2B',
  '410',
  'CEK',
  '2968',
  'OVB',
  '4078',
  None,
  '0',
  'CR2')]

### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest amount of active airlines?
- Which countries have the highest amount of inactive airlines?
- What about airports by timezones?

In [24]:
# Which countries have the highest amount of active airports?
cursor.execute("""
select country, count(active) as active
from airlines
where active = 'Y'
group by country order by active desc;
""")
print(*cursor.fetchall(), sep='\n')

('United States', 141)
('Russia', 72)
('United Kingdom', 40)
('Germany', 37)
('Canada', 34)
('Australia', 26)
('China', 25)
('Spain', 24)
('Brazil', 23)
('France', 22)
('Japan', 19)
('Italy', 18)
('Indonesia', 17)
('India', 17)
('Turkey', 16)
('Thailand', 16)
('Sweden', 15)
('Switzerland', 14)
('Ukraine', 13)
('Portugal', 13)
('Mexico', 12)
('Finland', 12)
('Egypt', 12)
('Austria', 12)
('Peru', 11)
('South Africa', 10)
('Greece', 10)
('Argentina', 10)
('Netherlands', 9)
('Iceland', 9)
('Colombia', 9)
('United Arab Emirates', 8)
('Singapore', 8)
('Romania', 8)
('Poland', 8)
('Philippines', 8)
('Vietnam', 7)
('Taiwan', 7)
('Pakistan', 7)
('Malaysia', 7)
('Ireland', 7)
('Iran', 7)
('Ecuador', 7)
('Denmark', 7)
('South Korea', 6)
('Norway', 6)
('Nepal', 6)
('Kazakhstan', 6)
('Israel', 6)
('Georgia', 6)
('Chile', 6)
('Burma', 6)
('Belgium', 6)
('Slovakia', 5)
('Republic of Korea', 5)
('Nigeria', 5)
('Morocco', 5)
('Lithuania', 5)
('Dominican Republic', 5)
('Czech Republic', 5)
('Cambodia', 

In [25]:
# Which countries have the highest amount of inactive airports?
cursor.execute("""
SELECT country, active
FROM (select country, count(active) as active
from airlines
where active = 'N'
group by country) order by active desc;
""")
print(*cursor.fetchmany(5), sep='\n')

('United States', 939)
('Mexico', 427)
('United Kingdom', 367)
('Canada', 284)
('Russia', 158)


In [26]:
# What about airports by timezones?
#a) grouping airports by timezone
#b) which timezones have the highest amount of active and inactive airports

cursor.execute("""
select  distinct timezone, ai.country, count(active) as active
from airlines ai
left join airports ap
on ai.icao = ap.icao
where active = 'N'
group by timezone
order by active desc;
 """)
print(*cursor.fetchall(), sep='\n')

('America/New_York', 'United States', 7590)
(None, 'United States', 4838)
('America/Anchorage', 'United States', 3772)
('Europe/Berlin', 'United States', 2944)
('Asia/Chongqing', 'United States', 2530)
('America/Los_Angeles', 'Canada', 2439)
('America/Chicago', 'Mexico', 2209)
('Europe/London', 'United States', 1932)
('\\N', 'United States', 1518)
('Indian/Maldives', 'United States', 1288)
('Europe/Paris', 'United States', 1104)
('America/Vancouver', 'United States', 1012)
('America/Toronto', 'United States', 1012)
('Europe/Moscow', 'United States', 966)
('Europe/Rome', 'United States', 828)
('Asia/Tokyo', 'United States', 736)
('America/Denver', 'United States', 736)
('Africa/Nairobi', 'United States', 736)
('Europe/Amsterdam', 'United States', 690)
('America/Cayenne', 'United States', 690)
('Pacific/Port_Moresby', 'United States', 644)
('Asia/Irkutsk', 'United States', 644)
('Europe/Zurich', 'United States', 552)
('Africa/Gaborone', 'United States', 552)
('Europe/Stockholm', 'United 

In [27]:
cursor.execute("""
select  count(icao) as active
from airlines
""")
print(*cursor.fetchall(), sep='\n')

(5961,)


In [28]:
cursor.execute("""
select name, timezone, count(*) 
from airports
group by timezone
order by count(*) desc;
 """)
print(*cursor.fetchall(), sep='\n')

('Putnam County Airport', 'America/New_York', 628)
('Door County Cherryland Airport', 'America/Chicago', 373)
('Bautzen', 'Europe/Berlin', 319)
('Tyonek Airport', 'America/Anchorage', 258)
('Calais Dunkerque', 'Europe/Paris', 232)
('Eastern Oregon Regional Airport', 'America/Los_Angeles', 226)
('Capital Intl', 'Asia/Chongqing', 222)
('Belfast Intl', 'Europe/London', 193)
('Sault Ste Marie', 'America/Toronto', 159)
('Ahmedabad', 'Asia/Calcutta', 141)
('Riverton Regional', 'America/Denver', 135)
('Narita Intl', 'Asia/Tokyo', 131)
('Aeropuerto de Rafaela', 'America/Cordoba', 104)
('Alexander Bay', 'Africa/Johannesburg', 104)
('Pulkovo', 'Europe/Moscow', 100)
('Campo Delio Jardim De Mattos', 'America/Sao_Paulo', 97)
('Amendola', 'Europe/Rome', 93)
('Guantanamo Bay Ns', '\\N', 92)
('Malmen', 'Europe/Stockholm', 86)
('Whitsunday Airstrip', 'Australia/Brisbane', 81)
('Abadan', 'Asia/Tehran', 81)
('Tofino', 'America/Vancouver', 78)
('Guvercinlik', 'Europe/Istanbul', 77)
('Melilla', 'Europe/Mad

In [29]:
#a) grouping airports by timezone
cursor.execute("""
select Distinct timezone , count(active)
from airlines left outer join airports ap on airlines.icao=ap.icao
where active = 'N'
group by timezone
order by active""")
print(*cursor.fetchall(), sep='\n')

(None, 4838)
('Africa/Addis_Ababa', 46)
('Africa/Algiers', 46)
('Africa/Bangui', 46)
('Africa/Casablanca', 46)
('Africa/Dar_es_Salaam', 322)
('Africa/Freetown', 46)
('Africa/Gaborone', 552)
('Africa/Harare', 92)
('Africa/Johannesburg', 460)
('Africa/Khartoum', 92)
('Africa/Lagos', 276)
('Africa/Luanda', 46)
('Africa/Lusaka', 138)
('Africa/Maputo', 138)
('Africa/Mogadishu', 92)
('Africa/Nairobi', 736)
('Africa/Ndjamena', 46)
('Africa/Nouakchott', 46)
('Africa/Tripoli', 138)
('Africa/Windhoek', 184)
('America/Anchorage', 3772)
('America/Belize', 368)
('America/Boa_Vista', 138)
('America/Bogota', 138)
('America/Caracas', 138)
('America/Cayenne', 690)
('America/Chicago', 2209)
('America/Cordoba', 506)
('America/Curacao', 46)
('America/Danmarkshavn', 46)
('America/Denver', 736)
('America/Edmonton', 506)
('America/Fortaleza', 138)
('America/Godthab', 92)
('America/Guatemala', 46)
('America/Guayaquil', 92)
('America/Guyana', 46)
('America/Halifax', 92)
('America/Havana', 46)
('America/Jamaica

In [30]:
# It's always a good idea to close our connections when we're done
cursor.close()
con.close()

## Goal 5: Transfering from sqlite to pandas

In [31]:
import pandas as pd

pd_con = sqlite3.connect("data/flights.db")
df = pd.read_sql_query("select * from airports limit 5;", pd_con)
df

index id                        name          city           country code  \
0      0  1                      Goroka        Goroka  Papua New Guinea  GKA   
1      1  2                      Madang        Madang  Papua New Guinea  MAG   
2      2  3                 Mount Hagen   Mount Hagen  Papua New Guinea  HGU   
3      3  4                      Nadzab        Nadzab  Papua New Guinea  LAE   
4      4  5  Port Moresby Jacksons Intl  Port Moresby  Papua New Guinea  POM   

   icao   latitude   longitude altitude offset dst              timezone  
0  AYGA  -6.081689  145.391881     5282     10   U  Pacific/Port_Moresby  
1  AYMD  -5.207083    145.7887       20     10   U  Pacific/Port_Moresby  
2  AYMH  -5.826789  144.295861     5388     10   U  Pacific/Port_Moresby  
3  AYNZ  -6.569828  146.726242      239     10   U  Pacific/Port_Moresby  
4  AYPY  -9.443383   147.22005      146     10   U  Pacific/Port_Moresby

**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

In [32]:
# Which countries have the highest amount of active airports?
sql = '''
SELECT country, active 
FROM (select country, count(active) as active
from airlines
where active = 'Y'
group by country) order by active desc;

'''
pd.read_sql_query(sql, pd_con)

country  active
0           United States     141
1                  Russia      72
2          United Kingdom      40
3                 Germany      37
4                  Canada      34
..                    ...     ...
190  Syrian Arab Republic       1
191                  Togo       1
192   Trinidad and Tobago       1
193          Turkmenistan       1
194               Vanuatu       1

[195 rows x 2 columns]

In [33]:
pd_con.close()

## Integration

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

Database information:<br>
- How many tables are in the database?
- What's the primary key of each table?
- What foreign keys join the tables together?
- If you had to draw a schema of how the tables are connected, what would it look like?

To answer the question:<br>
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

In [34]:
conn = sqlite3.connect('data/Chinook_Sqlite.sqlite')
cur = conn.cursor()

### How many tables are in the database?

In [35]:
# How many tables are in the database?

cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
len(cur.fetchall())

11

In [36]:
#the tables are:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
cur.fetchall()

[('Album',),
 ('Artist',),
 ('Customer',),
 ('Employee',),
 ('Genre',),
 ('Invoice',),
 ('InvoiceLine',),
 ('MediaType',),
 ('Playlist',),
 ('PlaylistTrack',),
 ('Track',)]

### What's the primary key of each table?

In [37]:
# What's the primary key of each table?

cur.execute("PRAGMA table_info(Album)")
cur.description

# Pk is the last element, can either reference it as .pk or read it

(('cid', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('type', None, None, None, None, None, None),
 ('notnull', None, None, None, None, None, None),
 ('dflt_value', None, None, None, None, None, None),
 ('pk', None, None, None, None, None, None))

Pk is the last element, can either reference it as .pk or read it

In [38]:
#code to find out just the primary key
cur.execute('SELECT l.name FROM pragma_table_info("Album") as l WHERE l.pk = 1;')
info1 = cur.fetchall()
print("Primany key for table Album =",info1)
cur.execute('SELECT l.name FROM pragma_table_info("Artist") as l WHERE l.pk = 1;')
info2 = cur.fetchall()
print("Primany key for table Artist =",info2)
cur.execute('SELECT l.name FROM pragma_table_info("Customer") as l WHERE l.pk = 1;')
info3 = cur.fetchall()
print("Primany key for table Customer =",info3)
cur.execute('SELECT l.name FROM pragma_table_info("Employee") as l WHERE l.pk = 1;')
info4 = cur.fetchall()
print("Primany key for table Employee =",info4)
cur.execute('SELECT l.name FROM pragma_table_info("Genre") as l WHERE l.pk = 1;')
info5 = cur.fetchall()
print("Primany key for table Genre =",info5)
cur.execute('SELECT l.name FROM pragma_table_info("Invoice") as l WHERE l.pk = 1;')
info6 = cur.fetchall()
print("Primany key for table Invoice =",info6)
cur.execute('SELECT l.name FROM pragma_table_info("InvoiceLine") as l WHERE l.pk = 1;')
info7 = cur.fetchall()
print("Primany key for table InvoiceLine =",info7)
cur.execute('SELECT l.name FROM pragma_table_info("MediaType") as l WHERE l.pk = 1;')
info8 = cur.fetchall()
print("Primany key for table MediaType =",info8)
cur.execute('SELECT l.name FROM pragma_table_info("PlayList") as l WHERE l.pk = 1;')
info9 = cur.fetchall()
print("Primany key for table PlayList =",info9)
cur.execute('SELECT l.name FROM pragma_table_info("PlaylistTrack") as l WHERE l.pk = 1;')
info10 = cur.fetchall()
print("Primany key for table PlaylistTrack =",info10)
cur.execute('SELECT l.name FROM pragma_table_info("Track") as l WHERE l.pk = 1;')
info11 = cur.fetchall()
print("Primany key for table Track =",info11)

Primany key for table Album = [('AlbumId',)]
Primany key for table Artist = [('ArtistId',)]
Primany key for table Customer = [('CustomerId',)]
Primany key for table Employee = [('EmployeeId',)]
Primany key for table Genre = [('GenreId',)]
Primany key for table Invoice = [('InvoiceId',)]
Primany key for table InvoiceLine = [('InvoiceLineId',)]
Primany key for table MediaType = [('MediaTypeId',)]
Primany key for table PlayList = [('PlaylistId',)]
Primany key for table PlaylistTrack = [('PlaylistId',)]
Primany key for table Track = [('TrackId',)]


In [39]:
#AlbumID
cur.execute('pragma table_info("Album")')
info1 = cur.fetchall()
print("Album",info1)

Album [(0, 'AlbumId', 'INTEGER', 1, None, 1), (1, 'Title', 'NVARCHAR(160)', 1, None, 0), (2, 'ArtistId', 'INTEGER', 1, None, 0)]


In [40]:
#ArtistID
cur.execute('pragma table_info("Artist")')
info2 = cur.fetchall()
print("Artist", info2)

Artist [(0, 'ArtistId', 'INTEGER', 1, None, 1), (1, 'Name', 'NVARCHAR(120)', 0, None, 0)]


In [41]:
#customerid
cur.execute('pragma table_info("Customer")')
info3 = cur.fetchall()
print("Customer", info3)

Customer [(0, 'CustomerId', 'INTEGER', 1, None, 1), (1, 'FirstName', 'NVARCHAR(40)', 1, None, 0), (2, 'LastName', 'NVARCHAR(20)', 1, None, 0), (3, 'Company', 'NVARCHAR(80)', 0, None, 0), (4, 'Address', 'NVARCHAR(70)', 0, None, 0), (5, 'City', 'NVARCHAR(40)', 0, None, 0), (6, 'State', 'NVARCHAR(40)', 0, None, 0), (7, 'Country', 'NVARCHAR(40)', 0, None, 0), (8, 'PostalCode', 'NVARCHAR(10)', 0, None, 0), (9, 'Phone', 'NVARCHAR(24)', 0, None, 0), (10, 'Fax', 'NVARCHAR(24)', 0, None, 0), (11, 'Email', 'NVARCHAR(60)', 1, None, 0), (12, 'SupportRepId', 'INTEGER', 0, None, 0)]


In [42]:
#EmployeeId
cur.execute('pragma table_info("Employee")')
info4 = cur.fetchall()
print("Employee", info4)

Employee [(0, 'EmployeeId', 'INTEGER', 1, None, 1), (1, 'LastName', 'NVARCHAR(20)', 1, None, 0), (2, 'FirstName', 'NVARCHAR(20)', 1, None, 0), (3, 'Title', 'NVARCHAR(30)', 0, None, 0), (4, 'ReportsTo', 'INTEGER', 0, None, 0), (5, 'BirthDate', 'DATETIME', 0, None, 0), (6, 'HireDate', 'DATETIME', 0, None, 0), (7, 'Address', 'NVARCHAR(70)', 0, None, 0), (8, 'City', 'NVARCHAR(40)', 0, None, 0), (9, 'State', 'NVARCHAR(40)', 0, None, 0), (10, 'Country', 'NVARCHAR(40)', 0, None, 0), (11, 'PostalCode', 'NVARCHAR(10)', 0, None, 0), (12, 'Phone', 'NVARCHAR(24)', 0, None, 0), (13, 'Fax', 'NVARCHAR(24)', 0, None, 0), (14, 'Email', 'NVARCHAR(60)', 0, None, 0)]


In [43]:
#GenreID
cur.execute('pragma table_info("Genre");')
info5 = cur.fetchall()
print("Genre", info5)

Genre [(0, 'GenreId', 'INTEGER', 1, None, 1), (1, 'Name', 'NVARCHAR(120)', 0, None, 0)]


In [44]:
#InvoiceId
cur.execute('pragma table_info("Invoice");')
info6 = cur.fetchall()
print("Invoice", info6)

Invoice [(0, 'InvoiceId', 'INTEGER', 1, None, 1), (1, 'CustomerId', 'INTEGER', 1, None, 0), (2, 'InvoiceDate', 'DATETIME', 1, None, 0), (3, 'BillingAddress', 'NVARCHAR(70)', 0, None, 0), (4, 'BillingCity', 'NVARCHAR(40)', 0, None, 0), (5, 'BillingState', 'NVARCHAR(40)', 0, None, 0), (6, 'BillingCountry', 'NVARCHAR(40)', 0, None, 0), (7, 'BillingPostalCode', 'NVARCHAR(10)', 0, None, 0), (8, 'Total', 'NUMERIC(10,2)', 1, None, 0)]


In [45]:
#InvoiceLineId
cur.execute('pragma table_info("InvoiceLine");')
info7 = cur.fetchall()
print("InvoiceLineID", info7)

InvoiceLineID [(0, 'InvoiceLineId', 'INTEGER', 1, None, 1), (1, 'InvoiceId', 'INTEGER', 1, None, 0), (2, 'TrackId', 'INTEGER', 1, None, 0), (3, 'UnitPrice', 'NUMERIC(10,2)', 1, None, 0), (4, 'Quantity', 'INTEGER', 1, None, 0)]


In [46]:
#MediaTypeId
cur.execute('pragma table_info("MediaType");')
info8 = cur.fetchall()
print("MediaType", info8)

MediaType [(0, 'MediaTypeId', 'INTEGER', 1, None, 1), (1, 'Name', 'NVARCHAR(120)', 0, None, 0)]


In [47]:
#PlaylistId
cur.execute('pragma table_info("PlayList");')
info9 = cur.fetchall()
print("PlayList", info9)

PlayList [(0, 'PlaylistId', 'INTEGER', 1, None, 1), (1, 'Name', 'NVARCHAR(120)', 0, None, 0)]


In [48]:
#PlayListId
cur.execute('pragma table_info("PlaylistTrack");')
info10 = cur.fetchall()
print("PlayListTrack", info10)

PlayListTrack [(0, 'PlaylistId', 'INTEGER', 1, None, 1), (1, 'TrackId', 'INTEGER', 1, None, 2)]


In [49]:
#TrackId
cur.execute('pragma table_info("Track");')
info11 = cur.fetchall()
print("Track", info11)

Track [(0, 'TrackId', 'INTEGER', 1, None, 1), (1, 'Name', 'NVARCHAR(200)', 1, None, 0), (2, 'AlbumId', 'INTEGER', 0, None, 0), (3, 'MediaTypeId', 'INTEGER', 1, None, 0), (4, 'GenreId', 'INTEGER', 0, None, 0), (5, 'Composer', 'NVARCHAR(220)', 0, None, 0), (6, 'Milliseconds', 'INTEGER', 1, None, 0), (7, 'Bytes', 'INTEGER', 0, None, 0), (8, 'UnitPrice', 'NUMERIC(10,2)', 1, None, 0)]


### What foreign keys join the tables together?

In [50]:
# What foreign keys join the tables together?
cur.execute('PRAGMA foreign_key_list(Album);')
cur.description

# your code here
# hint: use "PRAGMA foreign_key_list()"
#id          seq         table       from        to          on_update   on_delete   match 

(('id', None, None, None, None, None, None),
 ('seq', None, None, None, None, None, None),
 ('table', None, None, None, None, None, None),
 ('from', None, None, None, None, None, None),
 ('to', None, None, None, None, None, None),
 ('on_update', None, None, None, None, None, None),
 ('on_delete', None, None, None, None, None, None),
 ('match', None, None, None, None, None, None))

As can be seen above the 4th element is the foreign key linking to table that is the 3rd element by table key that is 5th element

In [51]:
info = cur.fetchall()
print(info)

[(0, 0, 'Artist', 'ArtistId', 'ArtistId', 'NO ACTION', 'NO ACTION', 'NONE')]


In [52]:
#no foreign key in Artist
cur.execute('PRAGMA foreign_key_list(Artist);')
info = cur.fetchall()
print(info)

[]


In [53]:
#foreign is SupportRepId for customer table
cur.execute('PRAGMA foreign_key_list(Customer);')
info = cur.fetchall()
print(info)

[(0, 0, 'Employee', 'SupportRepId', 'EmployeeId', 'NO ACTION', 'NO ACTION', 'NONE')]


In [54]:
#foreign is ReportsTo for Employee table
cur.execute('PRAGMA foreign_key_list(Employee);')
info = cur.fetchall()
print(info)

[(0, 0, 'Employee', 'ReportsTo', 'EmployeeId', 'NO ACTION', 'NO ACTION', 'NONE')]


In [55]:
#no foreign in Genre table
cur.execute('PRAGMA foreign_key_list(Genre);')
info = cur.fetchall()
print(info)

[]


In [56]:
#Foreign key is CustomerId in Invoice table
cur.execute('PRAGMA foreign_key_list(Invoice);')
info = cur.fetchall()
print(info)

[(0, 0, 'Customer', 'CustomerId', 'CustomerId', 'NO ACTION', 'NO ACTION', 'NONE')]


In [57]:
#no foreign key in InvoiceLineID table
cur.execute('PRAGMA foreign_key_list(InvoiceLineID);')
info = cur.fetchall()
print(info)


[]


In [58]:
#no foreign key in MediaType table
cur.execute('PRAGMA foreign_key_list(MediaType);')
info = cur.fetchall()
print(info)

[]


In [59]:
#no foreign key in PlayList table
cur.execute('PRAGMA foreign_key_list(PlayList);')
info = cur.fetchall()
print(info)

[]


In [60]:
#no foreign key in MediaType table
cur.execute('PRAGMA foreign_key_list(PlayListTrack);')
info = cur.fetchall()
print(info)

[(0, 0, 'Track', 'TrackId', 'TrackId', 'NO ACTION', 'NO ACTION', 'NONE'), (1, 0, 'Playlist', 'PlaylistId', 'PlaylistId', 'NO ACTION', 'NO ACTION', 'NONE')]


In [61]:
#Foreign keys are MediaTypeId, GenreId,  and AlbumId
cur.execute('PRAGMA foreign_key_list(Track);')
info = cur.fetchall()
print(info)

[(0, 0, 'MediaType', 'MediaTypeId', 'MediaTypeId', 'NO ACTION', 'NO ACTION', 'NONE'), (1, 0, 'Genre', 'GenreId', 'GenreId', 'NO ACTION', 'NO ACTION', 'NONE'), (2, 0, 'Album', 'AlbumId', 'AlbumId', 'NO ACTION', 'NO ACTION', 'NONE')]


In [62]:
#cur.execute('select * from invoice;')
#cur.fetchall()

In [63]:
#cur.execute('PRAGMA table_info(invoice)')
#cur.fetchall()

### What are the max and min dates in the Invoice table?

In [64]:
# What are the max and min dates in the Invoice table?
sql = '''
SELECT MIN(Invoicedate) as MinInvoiceDate, 
       MAX(Invoicedate) AS MaxinvoiceDate
FROM Invoice ;
'''
pd.read_sql_query(sql, conn)

MinInvoiceDate       MaxinvoiceDate
0  2009-01-01 00:00:00  2013-12-22 00:00:00

## What tables would you need to answer "what is your most popular track?"


Track table and invoice table can be used to find out the most popular track

## What values from each table?
Quantity sold can be used to find out which track is popular. 

In [65]:
# cur.execute('pragma table_info("Track");')
# info7 = cur.fetchall()
# print("Track", info7)

In [66]:
# cur.execute('pragma table_info("Album");')
# info7 = cur.fetchall()
# print("Track", info7)

In [67]:
# cur.execute('pragma table_info("Artist");')
# info7 = cur.fetchall()
# print("Track", info7)

In [68]:
# cur.execute('pragma table_info("InvoiceLine");')
# info7 = cur.fetchall()
# print("InvoiceLineID", info7)

### # Put it all together:
### You need to create a query that can rank tracks in term of popularity.

Price, name, trackid from track table and
Unit price and quantity from invoice table

In [69]:
import pandas as pd
# Put it all together:
# You need to create a query that can rank tracks in term of popularity.

sql = '''select name, Quantity, TotalValue from
(SELECT Name, sum(il.Quantity) as Quantity, sum(il.Quantity*il.unitprice) as TotalValue
FROM Track T INNER JOIN InvoiceLine il
    ON il.TrackId = T.TrackId group by Name) order by Quantity desc;
'''
pd.read_sql_query(sql, conn)

Name  Quantity  TotalValue
0                                         The Trooper         5        4.95
1                                            Eruption         4        3.96
2                                Hallowed Be Thy Name         4        3.96
3                                 Sure Know Something         4        3.96
4                             The Number Of The Beast         4        3.96
...                                               ...       ...         ...
1883                                      Água E Fogo         1        0.99
1884                                    Água de Beber         1        0.99
1885                                           É Fogo         1        0.99
1886  Étude 1, In C Major - Preludio (Presto) - Liszt         1        0.99
1887                                           Óculos         1        0.99

[1888 rows x 3 columns]

### Advanced: get the artist who sang the song!

In [70]:
# Advanced: get the artist who sang the song!
sql = '''
select artist.name as ArtistName, album.title as AlbumTitle, Track.Name as TrackName
from artist
     inner join album on album.artistid = artist.artistid
     inner join Track on Track.albumid = album.albumid
where Track.name = 'The Trooper'

'''
pd.read_sql_query(sql, conn)

ArtistName                       AlbumTitle    TrackName
0  Iron Maiden                  A Real Dead One  The Trooper
1  Iron Maiden                 Live After Death  The Trooper
2  Iron Maiden  Live At Donington 1992 (Disc 2)  The Trooper
3  Iron Maiden                    Piece Of Mind  The Trooper
4  Iron Maiden                Rock In Rio [CD1]  The Trooper

#### GROUP BY

- Group columns by similar values
- SELECT COUNT(id), city from students GROUP BY city

#### HAVING

- Use to apply filter AFTER a `GROUP BY` based on aggregate criteria 
- `WHERE` is applied for conditions prior to the `GROUP BY`, `HAVING` is applied afterwards

For example, if we had a table of student names and the courses they were taking, we could ask a question such as which classes have 3 or more students with the name Matt?

Such a query would look something like this:

```SQL
SELECT
  class,
  COUNT(student_name) AS number_of_alexes
FROM student_courses
WHERE student_name = "Alex"
GROUP BY 1
HAVING COUNT(student_name) >= 2;
```

In [71]:
conn = sqlite3.connect('data/tutorial.db')
c = conn.cursor()

In [72]:
c.execute("SELECT name FROM sqlite_master WHERE type='table';")

print(c.fetchall())

[('responses',)]


In [73]:
columns = [x[0] for x in c.execute('select * from responses').description]

In [74]:
columns

['submitted_ts',
 'name',
 'birthday',
 'hometown',
 'fav_food_1',
 'fav_food_2',
 'time_in_dc',
 'siblings_count']

### Questions
1. What are the names of all of the students?
2. Which student has the most siblings?
3. How many students are only children?
4. Which 3 students have lived in NYC the shortest amount of time?
5. How many students are native New Yorkers?
6. Do any two students have the same favorite food?


1. What are the names of all of the students.

In [75]:
sql = '''
SELECT name
FROM responses;
'''
pd.read_sql_query(sql, conn)

name
0            Stephen
1            Donna C
2              Muoyo
3         Vyjayanthi
4       Anesu Masube
5   Michael Pallante
6      Stuart Murphy
7      Darian Madere
8               Alex
9      Justin Fleury
10           Nate Lu
11              Nick
12          Jeff Lin
13             Chuck
14       Jill Carrie
15              Alex
16              Nimu

2. Which student has the most siblings?

In [76]:
sql = '''
select name, (siblings_count)
from responses
'''
pd.read_sql_query(sql, conn)

name  siblings_count
0            Stephen               2
1            Donna C               1
2              Muoyo               1
3         Vyjayanthi               3
4       Anesu Masube               5
5   Michael Pallante               2
6      Stuart Murphy               1
7      Darian Madere               3
8               Alex               1
9      Justin Fleury               1
10           Nate Lu               3
11              Nick              12
12          Jeff Lin               2
13             Chuck               3
14       Jill Carrie               1
15              Alex               1
16              Nimu               1

3. How many students are only children?
Ans: NO children are only children

In [77]:
sql = '''
select name, siblings_count
from responses
where siblings_count = 0
'''
pd.read_sql_query(sql, conn)

Empty DataFrame
Columns: [name, siblings_count]
Index: []

4. Which 3 students have lived in NYC the shortest amount of time? (How long has each lived in NYC?)
 <br /> 
Ans: time_nyc not provided, but time_dc is provided. Therefore to query for shortest time lived in DC

In [78]:
sql = '''
SELECT name, time_in_dc
FROM responses a 
WHERE  3 >= (SELECT count(time_in_dc)
FROM responses b 
WHERE b.time_in_dc <= a.time_in_dc )
ORDER BY a.time_in_dc DESC;
'''
pd.read_sql_query(sql, conn)

name  time_in_dc
0  Stephen         0.0
1    Muoyo         0.0
2     Alex         0.0

5. How many students are native New Yorkers?


In [79]:
sql = '''
select name, hometown
from responses
where hometown like '%N%Y'
'''
pd.read_sql_query(sql, conn)

name      hometown
0  Muoyo  Brooklyn, NY
1   Nimu     Nyack, NY

In [80]:
#for Washington DC
sql = '''
select name, hometown
from responses
where hometown like '%D%C'
'''
pd.read_sql_query(sql, conn)

name        hometown
0  Justin Fleury  Washington, DC
1        Nate Lu  Washington, DC

6. Do any two students have the same favorite food?

This problem employs the `Having` clause.  Be sure to review the difference between the where and having clause here. (Where filters apply before the group by clause and conditions following the having clause are filters applied after the group by on the resulting aggregate [statistics].) A useful example in doing so, could be to modify the question to something with an additional filtering criterion such as 'do any native new yorkers have the same favorite food?' This would force students to use a where clause prior to the group by to filter the results. Alternatively, see the question below for an alternative but related problem on favorite foods.

In [81]:
sql = '''
SELECT Name, fav_food_2 as fav_food
FROM responses 
WHERE upper(fav_food_2) IN (
SELECT upper(fav_food_2)
FROM responses
GROUP BY upper(fav_food_2)
HAVING COUNT(*) > 1)

Union

SELECT Name, fav_food_1 as fav_food
FROM responses 
WHERE upper(fav_food_1) IN (
SELECT upper(fav_food_1)
FROM responses
GROUP BY upper(fav_food_1)
HAVING COUNT(*) > 1)

Union

SELECT distinct Name, fav_food_1 as fav_food
FROM responses
WHERE fav_food_1 IN
(
SELECT fav_food_2
FROM responses
GROUP BY upper(fav_food_2)
HAVING COUNT(*) > 0
)

Union

SELECT distinct Name, fav_food_2 as fav_food
FROM responses
WHERE fav_food_2 IN
(
SELECT fav_food_1
FROM responses
GROUP BY upper(fav_food_1)
HAVING COUNT(*) > 0
)
'''
pd.read_sql_query(sql, conn)

name fav_food
0     Darian Madere    sushi
1           Donna C    sushi
2       Jill Carrie    Sushi
3       Jill Carrie     Thai
4  Michael Pallante    Pasta
5  Michael Pallante    Pizza
6             Muoyo     Thai
7           Stephen    pizza
8        Vyjayanthi    Pasta

In [82]:
# sql = ''' SELECT name, fav_food_1, fav_food_2
# FROM responses

# '''
# pd.read_sql_query(sql, conn)

## More Questions

What are the favorite foods of this classroom?

In [83]:
sql = '''
SELECT fav_food_1 as fav_food
FROM responses
WHERE fav_food_1 IS NOT NULL
UNION
SELECT fav_food_2
FROM responses
WHERE fav_food_2 IS NOT NULL;

'''
pd.read_sql_query(sql, conn)

fav_food
0   A single poached egg on a bed of arugula
1                                      Bacon
2                                      Bread
3                              Buffalo Wings
4                                Carrot cake
5                                     Coffee
6                                 Crab cakes
7                             Dark Chocolate
8                                       Grey
9                                  Ice cream
10                                    Indian
11                                   Lobster
12                                  Nigerian
13                                   Noodles
14                                     Pasta
15                                     Pizza
16                                 Pork Ribs
17                                      Saag
18                                     Sadza
19                                     Steak
20                                     Sushi
21                                      Thai
22                                  Zucchini
23                           beef stroganoff
24                                  brownies
25                                   lasagna
26                                     pizza
27                                    shrimp
28                                     sushi
29                                tater tots

Which student was born closest to the cohort's graduation date (5/29/20)?

In [84]:
sql = '''
select name, b from
   (      select name, min(birthday) as b from 'responses' where birthday >= '5/29'
    union select name, max(birthday) as b from 'responses' where birthday <= '5/29')
    order by abs('5/29'-b) limit 1;
'''
pd.read_sql_query(sql, conn)

name    b
0  Justin Fleury  5/5